In [1]:
import sys
sys.path.append("/media/shadowmotion/0CD113590CD11359/code/demo/smp/GuaranteedDelivery")
# sys.path.append("d:\\code\\demo\\smp\\GuaranteedDelivery")
import warnings
warnings.filterwarnings('ignore', category=ResourceWarning)

In [11]:
from elasticsearch import Elasticsearch

import pandas as pd
import requests
import uuid
import json
import glob
import re
import os

In [3]:
from flask import Flask, request, jsonify
from elasticsearch import Elasticsearch

app = Flask(__name__)
es = Elasticsearch(["http://localhost:50004"])

# 添加数据到Elasticsearch
@app.route('/add', methods=['POST'])
def add_data():
    content = request.json
    response = es.index(index="sample-index", body=content)
    return jsonify(response)

# 从Elasticsearch搜索数据
@app.route('/search', methods=['GET'])
def search_data():
    keyword = request.args.get('keyword')
    body = {
        "query": {
            "match": {
                "name": keyword
            }
        }
    }
    response = es.search(index="sample-index", body=body)
    return jsonify(response['hits']['hits'])



In [4]:
# 源文件夹
CRAWL_DIRECTORY = "../data/chatglm_llm_fintech_raw_dataset/allcrawl"

In [5]:
file_names = glob.glob(CRAWL_DIRECTORY + '/**', recursive=True)

In [6]:
file_names = [e for e in file_names if 'json' in e]

In [7]:
def load(file_name):
    with open(file_name, 'r') as f:
        data = json.load(f)
    return data 

In [8]:
crawl_data_list = [load(file_name) for file_name in file_names]

In [9]:
n = 0
error_data_list = []
for data in crawl_data_list:
    try:
        data["2019年报"]["SECURITY_NAME_ABBR"]
        # print(data["2019年报"]["SECURITY_NAME_ABBR"])
    except:
        error_data_list.append(data)
        n += 1

In [10]:
forrmated_crwal_data_dict = {}
for crawl_data in crawl_data_list:
    for year, report in crawl_data.items():
        company_name = report["SECURITY_NAME_ABBR"]

        forrmated_crwal_data_dict.setdefault(company_name, {})

        for k, v in report.items():
            forrmated_crwal_data_dict[company_name].setdefault(year, {})

            if k not in forrmated_crwal_data_dict[company_name][year]:
                forrmated_crwal_data_dict[company_name][year][k] = v
                continue
            if not v:
                continue
            forrmated_crwal_data_dict[company_name][year][k] = v

            

In [20]:
with open("../data/test_temp/all_crawl.json", "w") as f:
    json.dump(forrmated_crwal_data_dict, f, indent=2, ensure_ascii=False)

In [4]:
with open("../data/test_temp/all_crawl.json", "r") as f:
    res_data = json.load(f)

In [49]:
from datetime import datetime
from elasticsearch import Elasticsearch
es = Elasticsearch('http://localhost:50004')

doc = {
    'author': 'author_name',
    'text': 'Interensting content...',
    'timestamp': datetime.now(),
}
resp = es.index(index="test-index", id=1, document=doc)
print(resp['result'])

created


In [45]:

# 定义索引名称
index_name = 'company_reports'

document = {
    "公司名": "平安银行",
    "2021年报": {
        "属性名1": "属性值1",
        "属性名2": "属性值2"
    }
}

# 插入文档
doc_id = "1"  # 你可以为每个文档定义一个唯一ID
result = es.index(index=index_name, id=doc_id, document=document)

# 定义模糊查询的关键字
query_keyword = "平安*"

# 构建查询体
query_body = {
    "query": {
        "wildcard": {
            "公司名": query_keyword
        }
    }
}

# 执行搜索
result = es.search(index=index_name, body=query_body)

# 打印搜索结果
for hit in result['hits']['hits']:
    print(hit['_source'])

/tmp/ipykernel_6042/366071372.py:29: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  result = es.search(index=index_name, body=query_body)


In [53]:
index_name = '平安银行_2021年报'
result = es.index(index=index_name, document=res_data['平安银行']['2021年报'])

In [64]:
'*ST星源'.lower().replace('*', '_')

'_st星源'

In [73]:
res_data['平安银行']['2021年报']

{'SECUCODE': '000001.SZ',
 'SECURITY_CODE': '000001',
 'SECURITY_NAME_ABBR': '平安银行',
 'ORG_CODE': '10004085',
 'ORG_TYPE': '银行',
 'REPORT_DATE': '2021-12-31 00:00:00',
 'REPORT_TYPE': '年报',
 'REPORT_DATE_NAME': '2021年报',
 'SECURITY_TYPE_CODE': '058001001',
 'NOTICE_DATE': '2022-03-10 00:00:00',
 'UPDATE_DATE': '2023-03-09 00:00:00',
 'CURRENCY': 'CNY',
 '营业总收入': 169383000000,
 '营业总收入 同比': 10.3170468015,
 '营业收入': 169383000000,
 '营业收入 同比': 10.3170468015,
 '利息收入 其中:利息收入': None,
 '利息收入 其中:利息收入 同比': None,
 '已赚保费': None,
 '已赚保费 同比': None,
 '手续费及佣金收入 其中:手续费及佣金收入': 40190000000,
 '手续费及佣金收入 其中:手续费及佣金收入 同比': 1.80869389,
 '其他业务收入': 105000000,
 '其他业务收入 同比': -5.4054054054,
 '营业总收入其他项目': None,
 '营业总收入其他项目 同比': None,
 '营业总成本': 123398000000,
 '营业总成本 同比': 5.8002452136,
 '营业成本': None,
 '营业成本 同比': None,
 '利息支出 其中:利息支出': 93200000000,
 '利息支出 其中:利息支出 同比': 6.4692644253,
 '手续费及佣金支出 其中:手续费及佣金支出': 7128000000,
 '手续费及佣金支出 其中:手续费及佣金支出 同比': -27.376464595,
 '研发费用': None,
 '研发费用 同比': None,
 '退保金': None,
 '退保金 同比': Non

In [90]:
query_body = {
    "query": {
        "match_all": {}
    },
    "_source": ["营业*"]
}


index_name = '平安银行_2021年报'
result = es.search(index=index_name, body=query_body, size=1000)  # size可以根据需要调整

for hit in result['hits']['hits']:
    print(hit['_source'])


{'营业总收入': 169383000000, '营业总收入 同比': 10.3170468015, '营业收入': 169383000000, '营业收入 同比': 10.3170468015, '营业总收入其他项目': None, '营业总收入其他项目 同比': None, '营业总成本': 123398000000, '营业总成本 同比': 5.8002452136, '营业成本': None, '营业成本 同比': None, '营业总成本其他项目': None, '营业总成本其他项目 同比': None, '营业利润其他项目': None, '营业利润其他项目 同比': None, '营业利润平衡项目': None, '营业利润平衡项目 同比': None, '营业利润': 45985000000, '营业利润 同比': 24.5902083503, '营业总收入(元)': 169383000000, '营业总收入同比增长(%)': 10.3170468015, '营业总收入滚动环比增长(%)': 3.176623946201}


/tmp/ipykernel_6042/956960186.py:10: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  result = es.search(index=index_name, body=query_body, size=1000)  # size可以根据需要调整


In [89]:
result

ObjectApiResponse({'took': 5, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 1, 'relation': 'eq'}, 'max_score': 1.0, 'hits': [{'_index': '平安银行_2021年报', '_id': 'hbcD1IkBwus9EusfA3tK', '_score': 1.0, '_source': {'营业成本': None}}]}})

In [67]:

for i, company in enumerate(res_data.keys()):
    for year in res_data[company].keys():
        if year not in ["2019年报", "2020年报", "2021年报"]:
            continue
        print(f"ToES {i} {company}, {year}")
        formatted_index_name = company.lower().replace('*', '0_')
        index_name = f'{formatted_index_name}_{year}'
        result = es.index(index=index_name, document=res_data[company][year])
print(i)

平安银行 2021年报
平安银行 2020年报
平安银行 2019年报
ST星源 2021年报
ST星源 2020年报
ST星源 2019年报
*ST全新 2021年报
*ST全新 2020年报
*ST全新 2019年报
神州高铁 2021年报
神州高铁 2020年报
神州高铁 2019年报
中国宝安 2021年报
中国宝安 2020年报
中国宝安 2019年报
美丽生态 2021年报
美丽生态 2020年报
美丽生态 2019年报
沙河股份 2021年报
沙河股份 2020年报
沙河股份 2019年报
深中华A 2021年报
深中华A 2020年报
深中华A 2019年报
深科技 2021年报
深科技 2020年报
深科技 2019年报
ST深天 2021年报
ST深天 2020年报
ST深天 2019年报
飞亚达 2021年报
飞亚达 2020年报
飞亚达 2019年报
深圳能源 2021年报
深圳能源 2020年报
深圳能源 2019年报
富奥股份 2021年报
富奥股份 2020年报
富奥股份 2019年报
大悦城 2021年报
大悦城 2020年报
大悦城 2019年报
深桑达A 2021年报
深桑达A 2020年报
深桑达A 2019年报
神州数码 2021年报
神州数码 2020年报
神州数码 2019年报
中国天楹 2021年报
中国天楹 2020年报
中国天楹 2019年报
中集集团 2021年报
中集集团 2020年报
中集集团 2019年报
东旭蓝天 2021年报
东旭蓝天 2020年报
东旭蓝天 2019年报
中洲控股 2021年报
中洲控股 2020年报
中洲控股 2019年报
*ST泛海 2021年报
*ST泛海 2020年报
*ST泛海 2019年报
京基智农 2021年报
京基智农 2020年报
京基智农 2019年报
德赛电池 2021年报
德赛电池 2020年报
德赛电池 2019年报
深天马A 2021年报
深天马A 2020年报
深天马A 2019年报
华锦股份 2021年报
华锦股份 2020年报
华锦股份 2019年报
中金岭南 2021年报
中金岭南 2020年报
中金岭南 2019年报
农产品 2021年报
农产品 2020年报
农产品 2019年报
深圳华强 2021年报
深圳华强 2020年报
深圳华强 2019年

KeyboardInterrupt: 

In [46]:
for hit in result['hits']['hits']:
    print(hit['_source'])

In [12]:
len(res_data)

4374

In [94]:
with open("../data/chatglm_llm_fintech_raw_dataset/uuid.json", "r") as f:
    uuid_dict = json.load(f)

In [95]:
list(uuid_dict.keys())[:5]

['2020-01-21__江苏安靠智能输电工程科技股份有限公司__300617__安靠智电__2019年__年度报告',
 '2020-01-23__浙江海翔药业股份有限公司__002099__海翔药业__2019年__年度报告',
 '2020-02-08__焦点科技股份有限公司__002315__焦点科技__2019年__年度报告',
 '2020-02-14__平安银行股份有限公司__000001__平安银行__2019年__年度报告',
 '2020-02-20__上海家化联合股份有限公司__600315__上海家化__2019年__年度报告']

In [93]:
list(res_data.keys())[:5]

['平安银行', 'ST星源', '*ST全新', '神州高铁', '中国宝安']

In [96]:
map_dict = {}
for i in list(res_data.keys()):
    map_dict[i] = []
    for ii in list(uuid_dict.keys()):
        if i in ii:
            map_dict[i].append(ii)


In [108]:
reversed_map_dict = {value: key for key, values in map_dict.items() for value in values}

In [109]:
reversed_map_dict

{'2020-02-14__平安银行股份有限公司__000001__平安银行__2019年__年度报告': '平安银行',
 '2021-02-02__平安银行股份有限公司__000001__平安银行__2020年__年度报告': '平安银行',
 '2022-03-10__平安银行股份有限公司__000001__平安银行__2021年__年度报告': '平安银行',
 '2020-04-24__神州高铁技术股份有限公司__000008__神州高铁__2019年__年度报告': '神州高铁',
 '2021-04-20__神州高铁技术股份有限公司__000008__神州高铁__2020年__年度报告': '神州高铁',
 '2022-04-29__神州高铁技术股份有限公司__000008__神州高铁__2021年__年度报告': '神州高铁',
 '2020-04-29__中国宝安集团股份有限公司__000009__中国宝安__2019年__年度报告': '中国宝安',
 '2021-04-30__中国宝安集团股份有限公司__000009__中国宝安__2020年__年度报告': '中国宝安',
 '2022-03-31__中国宝安集团股份有限公司__000009__中国宝安__2021年__年度报告': '中国宝安',
 '2021-04-21__深圳美丽生态股份有限公司__000010__美丽生态__2019年__年度报告': '美丽生态',
 '2021-04-21__深圳美丽生态股份有限公司__000010__美丽生态__2020年__年度报告': '美丽生态',
 '2022-04-28__深圳美丽生态股份有限公司__000010__美丽生态__2021年__年度报告': '美丽生态',
 '2020-03-27__沙河实业股份有限公司__000014__沙河股份__2019年__年度报告': '沙河股份',
 '2021-03-27__沙河实业股份有限公司__000014__沙河股份__2020年__年度报告': '沙河股份',
 '2022-03-31__沙河实业股份有限公司__000014__沙河股份__2021年__年度报告': '沙河股份',
 '2020-04-24__深圳长城开发科技股份有限公司__000021__深科技__2019年__年度

In [106]:
for k, v in map_dict.items():
    if len(v) > 6:
        print(k)

机器人
中国石化
中国船舶
光电股份
生物股份
太平洋
中国石油
出版传媒
园林股份


In [107]:
map_dict['中国石化']

['2020-03-26__中国石化上海石油化工股份有限公司__600688__上海石化__2019年__年度报告',
 '2020-03-30__中国石油化工股份有限公司__600028__中国石化__2019年__年度报告',
 '2020-04-30__中国石化山东泰山石油股份有限公司__000554__泰山石油__2019年__年度报告',
 '2021-03-25__中国石化上海石油化工股份有限公司__600688__上海石化__2020年__年度报告',
 '2021-03-29__中国石油化工股份有限公司__600028__中国石化__2020年__年度报告',
 '2021-04-28__中国石化山东泰山石油股份有限公司__000554__泰山石油__2020年__年度报告',
 '2022-03-24__中国石化上海石油化工股份有限公司__600688__上海石化__2021年__年度报告',
 '2022-03-28__中国石油化工股份有限公司__600028__中国石化__2021年__年度报告',
 '2022-04-28__中国石化山东泰山石油股份有限公司__000554__泰山石油__2021年__年度报告']

In [99]:
set([len(v) for v in map_dict.values()])

{0, 1, 2, 3, 4, 5, 6, 9, 10, 11, 12, 13, 23, 38, 54}

In [100]:
[len(v) for v in map_dict.values()].count(54)

1

In [18]:
list(res_data.keys())[0]

'平安银行'

In [19]:
res_data[list(res_data.keys())[0]].keys()

dict_keys(['2021年报', '2020年报', '2019年报', '2022年报', '2018年报', '2017年报', '2016年报', '2015年报', '2014年报'])

In [110]:
res_data['平安银行']['2021年报'].keys()

dict_keys(['SECUCODE', 'SECURITY_CODE', 'SECURITY_NAME_ABBR', 'ORG_CODE', 'ORG_TYPE', 'REPORT_DATE', 'REPORT_TYPE', 'REPORT_DATE_NAME', 'SECURITY_TYPE_CODE', 'NOTICE_DATE', 'UPDATE_DATE', 'CURRENCY', '营业总收入', '营业总收入 同比', '营业收入', '营业收入 同比', '利息收入 其中:利息收入', '利息收入 其中:利息收入 同比', '已赚保费', '已赚保费 同比', '手续费及佣金收入 其中:手续费及佣金收入', '手续费及佣金收入 其中:手续费及佣金收入 同比', '其他业务收入', '其他业务收入 同比', '营业总收入其他项目', '营业总收入其他项目 同比', '营业总成本', '营业总成本 同比', '营业成本', '营业成本 同比', '利息支出 其中:利息支出', '利息支出 其中:利息支出 同比', '手续费及佣金支出 其中:手续费及佣金支出', '手续费及佣金支出 其中:手续费及佣金支出 同比', '研发费用', '研发费用 同比', '退保金', '退保金 同比', '赔付支出净额', '赔付支出净额 同比', '提取保险合同准备金净额', '提取保险合同准备金净额 同比', '保单红利支出', '保单红利支出 同比', '分保费用', '分保费用 同比', '其他业务成本', '其他业务成本 同比', '税金及附加', '税金及附加 同比', '销售费用', '销售费用 同比', '管理费用', '管理费用 同比', '财务费用', '财务费用 同比', '利息费用 其中:利息费用', '利息费用 其中:利息费用 同比', '资产减值损失', '资产减值损失 同比', '信用减值损失', '信用减值损失 同比', '营业总成本其他项目', '营业总成本其他项目 同比', '公允价值变动收益 加:公允价值变动收益', '公允价值变动收益 加:公允价值变动收益 同比', '投资收益 加:投资收益', '投资收益 加:投资收益 同比', '其中:对联营企业和合营企业的投资收益/（损失）', '其中:对联营企业和合营企业的投资收益/（损失